## Dataset generation using opencv

In [1]:
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.utils import to_categorical
import keras
import keras_metrics
import matplotlib.pyplot as plt
import os 
from random import shuffle
from tqdm import tqdm

C:\Users\Marhaba\Anaconda3\envs\New_Environment_For_Keras\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Marhaba\Anaconda3\envs\New_Environment_For_Keras\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Marhaba\Anaconda3\envs\New_Environment_For_Keras\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\User

In [ ]:
from numba import cuda 
device = cuda.get_current_device()
print(device)
device.reset()

In [2]:
#AlexNet Architecture for EC estimation
model = keras.models.Sequential([
    keras.layers.Conv2D(filters=32, kernel_size=(3,3), strides=(1,1), activation='relu', input_shape=(250,250,1)),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Conv2D(filters=64, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Conv2D(filters=128, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Conv2D(filters=64, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Conv2D(filters=32, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(1048, activation='relu'),
    keras.layers.Dropout(0.8),
    keras.layers.Dense(4, activation='softmax')
])
model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 248, 248, 32)      320       
_________________________________________________________________
batch_normalization_1 (Batch (None, 248, 248, 32)      128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 123, 123, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 123, 123, 64)      51264     
_________________________________________________________________
batch_normalization_2 (Batch (None, 123, 123, 64)      256       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 61, 61, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 61, 61, 128)     

In [ ]:
def dataset_generation():
    FaceClassifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    def face_detection(image):
        gray_scale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        detected_face = FaceClassifier.detectMultiScale(gray_scale, 1.3, 5)
        
        if detected_face is ():
            return None
        for (x,y,w,h) in detected_face:
            face = image[y:y+h,x:x+w]
        return face
    
    videoCapture = cv2.VideoCapture(0)
    image_id = 0
    
    while True:
        ret, frame = videoCapture.read()
        if face_detection(frame) is not None:
            image_id+=1
            face = cv2.resize(face_detection(frame), (250,250))
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
            #Saving images from web cam to local disk
            file_name_path = "Dataset/"+"Yousaf_"+str(image_id)+'.jpg'
            cv2.imwrite(file_name_path, face)
            cv2.putText(face, str(image_id), (50,50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2 )
            cv2.imshow("Cropped_Face", face)
            if cv2.waitKey(1)==13 or int(image_id)==1000:
                break
                
    videoCapture.release()
    cv2.destroyAllWindows()
    print("Data Collection is completed")

In [ ]:
dataset_generation()

In [5]:
import numpy as np

In [6]:
#Creating labels for each person
def label(image):
    name = image.split("_")[0]
    #One hot encoding for four persons 
    if name == "Yousaf":
        return 0
    elif name == "Qazi":
        return 1
    elif name == "Abdullah":
        return 2
    elif name == "Manzoor":
        return 3

## Preparing dataset for deep learning model

In [7]:
def load_data():
    dataset = []
    for image_name in tqdm(os.listdir("C:\\Users\\Marhaba\\Videos\\FaceID-Door-Lock-System\\Source_Code\\Dataset")):
        image_path = os.path.join("C:\\Users\\Marhaba\\Videos\\FaceID-Door-Lock-System\\Source_Code\\Dataset",image_name)
        image = cv2.imread(image_path,cv2.IMREAD_GRAYSCALE)/256
        dataset.append([image,label(image_name)])
    shuffle(dataset)
    return dataset

In [8]:
dataset = load_data()

100%|██████████| 2500/2500 [00:03<00:00, 812.22it/s]


In [9]:
dataset[1]

[array([[0.15625   , 0.15234375, 0.1484375 , ..., 0.51953125, 0.51953125,
         0.515625  ],
        [0.15234375, 0.1484375 , 0.1484375 , ..., 0.5234375 , 0.51953125,
         0.51953125],
        [0.15234375, 0.1484375 , 0.1484375 , ..., 0.52734375, 0.5234375 ,
         0.5234375 ],
        ...,
        [0.21484375, 0.22265625, 0.23046875, ..., 0.53125   , 0.53125   ,
         0.53125   ],
        [0.22265625, 0.23046875, 0.23828125, ..., 0.53125   , 0.52734375,
         0.52734375],
        [0.23046875, 0.23828125, 0.24609375, ..., 0.53125   , 0.52734375,
         0.52734375]]),
 0]

In [10]:
#Splitting dataset into training and testing 
train_dataset = dataset[:1600]
test_dataset = dataset[1600::]
X = np.array([k[0] for k in train_dataset]).reshape(-1,250,250,1)
print(X.shape)
Y = [j[1] for j in train_dataset]
Y = to_categorical(Y)
X_test = np.array([k[0] for k in test_dataset]).reshape(-1,250,250,1)
print(X_test.shape)
Y_test = [j[1] for j in test_dataset]
Y_test = to_categorical(Y_test)


(1600, 250, 250, 1)
(900, 250, 250, 1)


## CNN architecture for face recognition

In [11]:
print(Y[0])

[0. 0. 0. 1.]


In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy',keras_metrics.precision(),keras_metrics.recall(),keras_metrics.f1_score()])
history=model.fit(X, Y, epochs=17, batch_size=32, verbose=1, validation_split=0.3)

tracking <tf.Variable 'Variable:0' shape=() dtype=int32> tp
tracking <tf.Variable 'Variable_1:0' shape=() dtype=int32> fp
tracking <tf.Variable 'Variable_2:0' shape=() dtype=int32> tp
tracking <tf.Variable 'Variable_3:0' shape=() dtype=int32> fn
tracking <tf.Variable 'Variable_4:0' shape=() dtype=int32> tp
tracking <tf.Variable 'Variable_5:0' shape=() dtype=int32> fp
tracking <tf.Variable 'Variable_6:0' shape=() dtype=int32> tp
tracking <tf.Variable 'Variable_7:0' shape=() dtype=int32> fn

Train on 1120 samples, validate on 480 samples
Epoch 1/17
 288/1120 [======>.......................] - ETA: 7:49 - loss: 1.8128 - accuracy: 0.6875 - precision: 0.7668 - recall: 0.7592 - f1_score: 0.7593

In [ ]:
model.evaluate(X_test,Y_test,verbose=1)

In [ ]:
history.history.keys()


In [ ]:
print(history.history.keys())
plt.plot(history.history['precision'])
plt.plot(history.history['recall'])
plt.plot(history.history['f1_score'])
plt.title('Precision Recall')
plt.ylabel('Precision')
plt.xlabel('Number of epocs')
plt.legend(['Precision', 'Recall','F1 Score'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['loss','val_loss'], loc='upper left')
plt.show()

In [ ]:
# saving and loading the model weights
 
# save model
model.save_weights('faceRecognitionModel')
print('Model Saved!')
 
# load model
savedModel = model.load_weights('faceRecognitionModel')
print('Model Loaded!')

In [ ]:
#AlexNet Architecture for EC estimation
model2 = keras.models.Sequential([
    keras.layers.Conv2D(filters=32, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(250,250,1)),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Conv2D(filters=64, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(1048, activation='relu'),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(4, activation='softmax')
])
model2.summary()
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy',keras_metrics.precision(),keras_metrics.recall(),keras_metrics.f1_score()])


In [ ]:
# load model
savedModel = model2.load_weights('faceRecognitionModel')
print('Model Loaded!')

In [ ]:
import tensorflow as tf
tf.__version__


In [ ]:
model2.evaluate(X_test,Y_test,verbose=1)

In [ ]:
def prediction_model():
    FaceClassifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    def face_detection(image):
        gray_scale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        detected_face = FaceClassifier.detectMultiScale(gray_scale, 1.3, 5)
        
        if detected_face is ():
            return None
        for (x,y,w,h) in detected_face:
            face = image[y:y+h,x:x+w]
        return face
    
    videoCapture = cv2.VideoCapture(0)    
    while True:
        ret, frame = videoCapture.read()
        if face_detection(frame) is not None:
            face = cv2.resize(face_detection(frame), (250,250))
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
            cv2.putText(face, str("Recognising"), (50,50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2 )
            cv2.imshow("Cropped_Face", face)
            prediction = model2.predict(face.reshape(-1, 250, 250, 1))
            print(prediction)
            if cv2.waitKey(1)==13:
                break
                
    videoCapture.release()
    cv2.destroyAllWindows()
    print("Data Collection is completed")

In [ ]:
prediction = model2.predict_classes(X_test[90].reshape(-1, 250, 250, 1))

In [ ]:
print(prediction)
print(Y_test[90])

In [ ]:
prediction_model()